### Codificación de las variables categoricas, escalado y modelo

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
# !pip install category_encoders
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
    silhouette_score, recall_score, precision_score, make_scorer, \
    roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
    classification_report, confusion_matrix

from sklearn import metrics


In [2]:
df_fraud_train = pd.read_csv("../data/train_pd_data_preprocessing_missing_outlier.csv")
df_fraud_test = pd.read_csv("../data/test_pd_data_preprocessing_missing_outlier.csv") 

In [3]:
df_fraud_train.columns

Index(['Unnamed: 0', 'device_os', 'source', 'housing_status',
       'employment_status', 'payment_type', 'fraud_bool',
       'prev_address_months_count_input', 'bank_branch_count_8w_input',
       'credit_risk_score_input', 'velocity_24h_input', 'income_input',
       'current_address_months_count_input', 'bank_months_count_input',
       'zip_count_4w_input', 'customer_age_input', 'month_input',
       'keep_alive_session_input', 'device_fraud_count_input',
       'device_distinct_emails_8w_input', 'velocity_4w_input',
       'velocity_6h_input', 'email_is_free_input',
       'date_of_birth_distinct_emails_4w_input', 'foreign_request_input',
       'name_email_similarity_input', 'phone_mobile_valid_input',
       'proposed_credit_limit_input', 'phone_home_valid_input',
       'intended_balcon_amount_input', 'session_length_in_minutes_input',
       'has_other_cards_input', 'days_since_request_input'],
      dtype='object')

In [4]:
df_fraud_train.dtypes

Unnamed: 0                                  int64
device_os                                  object
source                                     object
housing_status                             object
employment_status                          object
payment_type                               object
fraud_bool                                  int64
prev_address_months_count_input           float64
bank_branch_count_8w_input                float64
credit_risk_score_input                   float64
velocity_24h_input                        float64
income_input                              float64
current_address_months_count_input        float64
bank_months_count_input                   float64
zip_count_4w_input                        float64
customer_age_input                        float64
month_input                               float64
keep_alive_session_input                  float64
device_fraud_count_input                  float64
device_distinct_emails_8w_input           float64


## Codificación de la variable objetivo
1-- Verdadero y 0-- Falso

In [5]:
df_fraud_train['fraud_bool'].value_counts()

fraud_bool
0    791177
1      8823
Name: count, dtype: int64

Se cuenta la frequencia con la obtenemos los valores unicos de 0 y 1 en fraud_bool.

In [6]:
print(df_fraud_train['fraud_bool'].unique())

[0 1]


In [7]:
df_fraud_train['fraud_bool'].value_counts()

fraud_bool
0    791177
1      8823
Name: count, dtype: int64

In [8]:
X_train = df_fraud_train.drop('fraud_bool',axis=1)
X_test = df_fraud_test.drop('fraud_bool',axis=1)
y_train = df_fraud_train['fraud_bool']
y_test = df_fraud_test['fraud_bool']

Dividimos el conjunto de datos para posteriormente entrenarlo.

## Codificación del resto de variables categoricas
Por ejemplo, elijo codificarlas con one-hot-encoding todas. Se podría elegir codificar algunas de una forma y otras de otra. Por ejemplo, variables con pocas categorías con one hot encoding y variables con más categorias con TargetEncoding

In [9]:
list_columns_cat = list(X_train.select_dtypes("object", "category").columns)
list_other = list(set(X_train.columns)-set(list_columns_cat))

Generamos dos listas, una con variables categoricas y otra  con el resto.

In [10]:
ohe = ce.OneHotEncoder(cols=list_columns_cat)
model = ohe.fit(X_train, y_train)

Utiliza la biblioteca category_encoders para realizar la codificación one-hot de las variables categóricas en el conjunto de entrenamiento X_train.

In [11]:
model

OneHotEncoder(cols=['device_os', 'source', 'housing_status',
                    'employment_status', 'payment_type'])

In [12]:
X_train_t = model.transform(X_train, y_train)
X_test_t = model.transform(X_test, y_test)

Se aplica la transformación a los conjuntos de entrenamiento y prueba. 

In [13]:
len(list(X_train_t.columns))

53

In [14]:
X_train_t.dtypes.to_dict()

{'Unnamed: 0': dtype('int64'),
 'device_os_1': dtype('int64'),
 'device_os_2': dtype('int64'),
 'device_os_3': dtype('int64'),
 'device_os_4': dtype('int64'),
 'device_os_5': dtype('int64'),
 'source_1': dtype('int64'),
 'source_2': dtype('int64'),
 'housing_status_1': dtype('int64'),
 'housing_status_2': dtype('int64'),
 'housing_status_3': dtype('int64'),
 'housing_status_4': dtype('int64'),
 'housing_status_5': dtype('int64'),
 'housing_status_6': dtype('int64'),
 'housing_status_7': dtype('int64'),
 'employment_status_1': dtype('int64'),
 'employment_status_2': dtype('int64'),
 'employment_status_3': dtype('int64'),
 'employment_status_4': dtype('int64'),
 'employment_status_5': dtype('int64'),
 'employment_status_6': dtype('int64'),
 'employment_status_7': dtype('int64'),
 'payment_type_1': dtype('int64'),
 'payment_type_2': dtype('int64'),
 'payment_type_3': dtype('int64'),
 'payment_type_4': dtype('int64'),
 'payment_type_5': dtype('int64'),
 'prev_address_months_count_input': d

Genera un diccionario que muestra los tipos de datos de cada columna en el DataFrame X_train_t después de aplicar la transformación one-hot. 